# Simplex Algorithm in Numpy

---



---



#### Import numpy

In [ ]:
import numpy as np

#### Given the  following optimization problem
#### $\hspace{2.7cm} \min c^T\textbf{x}$

#### subject to $\hspace{.1cm} A_{eq}\textbf{x} = \textbf{b}_{eq}$
#### $\hspace{2.7cm} A_u \textbf{x} \leq \textbf{b}_u$
#### $\hspace{3.5cm}  \textbf{x} \geq \textbf{0}$

#### The simplex algorithm as class object will be initialize with

#### c: cost vector $c^T$
#### A: $A_{eq}$
#### b: $b_{eq}$
#### A_l: $A_{u}$
#### b_l: $b_{u}$

In [ ]:
class SimplexProblem():

    def __init__(self, c=None, A=None, b=None, A_l=None, b_l = None):

        if (A_l is None and  not (A is None) ):

            self.b = b
            self.artf_num = A.shape[0]
            z = np.eye(self.artf_num)
            self.A = np.concatenate((A,z), axis = 1)
            self.c = np.concatenate((c, np.ones((self.artf_num))), axis=0)
            self.slack_num = 0

            self.num_var = self.A.shape[1]

        elif (A is None and  not (A_l is None)):

            self.slack_num = A_l.shape[0]
            s = np.eye(self.slack_num)
            self.A = np.concatenate((A_l,s), axis = 1)
            self.b=b_l
            self.artf_num = 0
            self.c = np.concatenate((c, np.zeros((self.slack_num))), axis=0)

            self.num_var = self.A.shape[1]

        elif (not (A is None) and  not (A_l is None)):

            self.slack_num = A_l.shape[0]
            s = np.eye(self.slack_num)
            s_ = np.zeros( (A.shape[0], self.slack_num) )
            self.artf_num = A.shape[0]
            z = np.eye(self.artf_num)
            z_ =  np.zeros( (A_l.shape[0], self.artf_num) )
            A_ = np.concatenate((A, z), axis=1)
            A_ = np.concatenate((A_, s_), axis=1)
            A_l_ = np.concatenate((A_l, z_), axis=1)
            A_l_ = np.concatenate((A_l_, s), axis=1)
            self.A = np.concatenate((A_,A_l_), axis=0)
            self.b = np.concatenate((b,b_l), axis=0)
            self.c = np.concatenate((c, 1000000*np.ones((self.artf_num)),
                                     np.zeros(self.slack_num)), axis=0)

            self.num_var = self.A.shape[1]


    def solve(self):

        x_b, x_n, x_b_val = self.init_base()
        opt = False
        z = 0

        ix=1
        while not opt:

            x_b, x_n, x_b_val, z, opt, status = self.iterate(x_b,x_n)

            ix+=1

        return x_b, x_n, x_b_val, z, status

    def init_base(self):

        x_b = [i for i in range(self.num_var - (self.artf_num+self.slack_num),
                                self.num_var )]
        x_n = [i for i in range(0, self.num_var - (self.artf_num+self.slack_num))]

        x_b_val= self.b

        return x_b, x_n, x_b_val

    def iterate(self,x_b,x_n):

        B    = np.zeros((self.A.shape[0],self.A.shape[0]))
        N    = np.zeros((self.A.shape[0],self.A.shape[1]-self.A.shape[0]))
        c_b = np.zeros((self.A.shape[0]))
        c_n = np.zeros((self.A.shape[1]-self.A.shape[0]))



        idx=0

        for var in x_b:
            B[:,idx] = self.A[:,var]
            c_b[idx] = self.c[var]
            idx+=1

        idx=0
        for var in x_n:
            N[:,idx] = self.A[:,var]
            c_n[idx] = self.c[var]
            idx+=1

        B_inv = np.linalg.inv(B)
        dnT    = np.subtract(c_n,np.matmul( c_b, np.matmul(B_inv,N) ))
        y        = np.matmul(c_b,B_inv)
        N_nu = np.matmul(B_inv,N)

        x_in        = x_n[np.argmin(dnT)]
        col_x_in = N_nu[:,np.argmin(dnT)]

        propor = np.divide(np.matmul(B_inv,self.b),
                           np.clip(col_x_in,0.0000000001,np.infty))

        x_out = x_b[np.argmin(propor)]

        x_b[np.argmin(propor)] = x_in
        x_n[np.argmin(dnT)]      = x_out

        x_b_val = np.matmul(B_inv,self.b)

        z = -1*np.matmul(y,self.b)


        dnT_check        = [1 if x >0 else 0 for x in dnT ]
        col_x_in_check = [1 if x < 0 else 0 for x in col_x_in ]

        if  all(dnT_check):
            opt = True
            status = 'Optimum'
        elif all(col_x_in_check):
            opt = True
            status = 'Unboundend'
        else:
            opt = False
            status='Not optimal'

        return x_b, x_n, x_b_val,z, opt, status

#### Lets define the linear optimization problem:
$\min \hspace{0.5cm} 4x_1+x_2\\
\hspace{1.5cm} 3x_1+x_2 = 3\\
 \hspace{0.6cm}-4x_1-3x_2 \leq -6\\
\hspace{1.5cm} x_1+2x_2 \leq 4\\
\hspace{2.5cm}  \textbf{x} \geq \textbf{0}
$

In [ ]:
c     =[4,1]
A    = np.array([[3,1]])
b    = [3]
A_l = np.array([[-4,-3],[1,2]])
b_l =[-6,4]

#### Run simplex algorithm

In [ ]:
modelo = SimplexProblem(c=c,A_l=A_l,b_l=b_l, A= A, b=b)
x_b, x_n, x_b_val, z, status = modelo.solve()

#### Show results

In [ ]:
for i in range(len(x_b)):
  print("Variable: x_{} , value: {} ".format(str(x_b[i]+1 ),x_b_val[i]  ))

Variable: x_2 , value: 1.8000000000000003 
Variable: x_1 , value: 0.40000000000000013 
Variable: x_5 , value: 1.0000000000000009 
